In [1]:
#https://towardsdatascience.com/generating-new-faces-with-variational-autoencoders-d13cfcb5f0a8
%env HSA_OVERRIDE_GFX_VERSION=10.3.0
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env ROCM_PATH=/opt/rocm
import tensorflow as tf
tf.test.is_built_with_rocm()
# document export HSA_OVERRIDE_GFX_VERSION=10.3.0
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

env: HSA_OVERRIDE_GFX_VERSION=10.3.0
env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: ROCM_PATH=/opt/rocm


2023-03-20 10:04:10.949222: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 10:04:11.092137: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-20 10:04:12.223885: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-20 10:04:12.238487: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] suc

In [2]:
import numpy as np

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint 
from keras.utils import plot_model
import keras

In [3]:
vae_model = keras.models.load_model("vae_model_celeba.h5", compile=False)
vae_decoder = keras.models.load_model("vae_decoder_celeba.h5", compile=False)
vae_encoder = keras.models.load_model("vae_encoder_celeba.h5", compile=False)

2023-03-20 10:04:22.878294: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 10:04:22.879052: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-20 10:04:22.879174: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-20 10:04:22.879220: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), bu

In [4]:
vae_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 encoder_conv_0 (Conv2D)        (None, 64, 64, 32)   896         ['encoder_input[0][0]']          
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 64, 64, 32)   0           ['encoder_conv_0[0][0]']         
                                                                                                  
 encoder_conv_1 (Conv2D)        (None, 32, 32, 64)   18496       ['leaky_re_lu[0][0]']      

In [13]:
#TODO document jupyter
import pickle
import configparser
import sqlite3 as sl
import pandas as pd
import numpy as np
from PIL import Image

configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)
datasetPathDatabase =  configParser.get('COMMON', 'datasetPathDatabase') + '/dataset.db'
p =  configParser.get('extractFacesLatents', 'dbChunk')

# TODO Better display of progress and handling of exceptions
contLoop = True # Flag to continue to get chunks of videos from database
con = sl.connect(datasetPathDatabase)
offset = 0
while(contLoop):
    data = con.execute("SELECT ID,FACE_PATH FROM FACE WHERE LATENT_REP IS NULL ORDER BY ID ASC LIMIT " + p + " OFFSET " + str(offset))
    contLoop = False
    offset = offset + int(p)
    print("Got chunk of faces from database. Extracting latent features...")
    dataGotten = data.fetchall()
    for id,facePath in dataGotten:
        contLoop = True
        face_img = Image.open(facePath)
        face_img.load() # required for png.split()
        face_img2 = Image.new("RGB", face_img.size, (255, 255, 255))
        face_img2.paste(face_img, mask=face_img.split()[3]) # 3 is the alpha channel
        face_img = face_img2
        face_img = np.array(face_img)
        face_img = np.expand_dims(face_img,0)
        latent_rep = vae_encoder.predict(face_img)
        latent_rep = latent_rep[0]
        latent_rep_pickled = pickle.dumps(latent_rep)

        # Insert latent representations into database
        sql = '''UPDATE FACE SET LATENT_REP = ? WHERE ID = ?'''

        
        cur = con.cursor()
        data = [latent_rep_pickled,id]
        cur.execute(sql, data)
        con.commit()
        cur.close()


con.close()
        
        





Got chunk of faces from database. Extracting latent features...
Got chunk of faces from database. Extracting latent features...
Got chunk of faces from database. Extracting latent features...
Got chunk of faces from database. Extracting latent features...
Got chunk of faces from database. Extracting latent features...
Got chunk of faces from database. Extracting latent features...
Got chunk of faces from database. Extracting latent features...
Got chunk of faces from database. Extracting latent features...
Got chunk of faces from database. Extracting latent features...
Got chunk of faces from database. Extracting latent features...
Got chunk of faces from database. Extracting latent features...
Got chunk of faces from database. Extracting latent features...
Got chunk of faces from database. Extracting latent features...
Got chunk of faces from database. Extracting latent features...
Got chunk of faces from database. Extracting latent features...
Got chunk of faces from database. Extrac

KeyboardInterrupt: 

In [44]:
dataGotten

[(1,
  '/home/gamal/Datasets/Dataset1/Faces/NZOCucECvDs_62.732333_65.732333_face_1.png'),
 (2,
  '/home/gamal/Datasets/Dataset1/Faces/NZOCucECvDs_62.732333_65.732333_face_2.png'),
 (3,
  '/home/gamal/Datasets/Dataset1/Faces/NZOCucECvDs_62.732333_65.732333_face_3.png'),
 (4,
  '/home/gamal/Datasets/Dataset1/Faces/1uizi6R-PMU_116.920000_120.000000_face_1.png'),
 (5,
  '/home/gamal/Datasets/Dataset1/Faces/1uizi6R-PMU_116.920000_120.000000_face_2.png'),
 (6,
  '/home/gamal/Datasets/Dataset1/Faces/1uizi6R-PMU_116.920000_120.000000_face_3.png'),
 (7,
  '/home/gamal/Datasets/Dataset1/Faces/4ipH_P0io9o_79.160000_82.480000_face_1.png'),
 (8,
  '/home/gamal/Datasets/Dataset1/Faces/4ipH_P0io9o_79.160000_82.480000_face_2.png'),
 (9,
  '/home/gamal/Datasets/Dataset1/Faces/4ipH_P0io9o_79.160000_82.480000_face_3.png'),
 (10,
  '/home/gamal/Datasets/Dataset1/Faces/x2brYLv1HJk_146.533000_149.933000_face_1.png'),
 (11,
  '/home/gamal/Datasets/Dataset1/Faces/x2brYLv1HJk_146.533000_149.933000_face_2.png')

In [40]:
latents = vae_encoder.predict(image)

In [41]:
latents.shape

(126, 200)

In [42]:
latents[0]

array([ 1.0843052 , -0.1574214 ,  0.96014076, -0.01608606,  0.37075156,
        2.7769132 , -1.194191  ,  0.8174098 , -1.2320787 , -0.26595482,
        1.5342367 ,  1.5997851 , -1.3255134 ,  0.51969504,  0.38224915,
        1.715023  , -0.20995013,  0.4509715 , -0.34549168, -1.4251034 ,
       -0.7809226 , -0.7464825 , -1.8087181 ,  0.03291734, -0.66174364,
       -0.5666951 , -1.0904781 , -0.17445779, -0.29315504,  1.0310547 ,
       -0.40406942,  1.0277227 , -0.31162962,  1.1208587 , -0.4127219 ,
       -0.8519272 , -0.2411303 , -2.1992898 ,  0.7975864 , -0.34818193,
        1.8494238 , -1.5860696 ,  1.528455  , -1.1227158 ,  0.7889575 ,
        4.2257195 , -2.103981  , -0.27558464, -0.9730452 , -2.1764283 ,
       -1.2047594 ,  1.3019958 , -0.08131307,  0.9376823 , -1.6207539 ,
        1.3475494 ,  1.0325551 ,  0.25274947, -0.34661037,  0.23892787,
       -0.13960503, -0.01966438,  0.69003373, -1.181275  ,  0.77673805,
        0.09425893, -2.7011867 , -0.5522225 , -1.0743376 , -1.15